In [2]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import random
from mesa import Agent, Model
import mesa.time as time
import mesa.space as space
from mesa.datacollection import DataCollector
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [7]:
class Ambulance (Agent):
    def __init__(self, unique_id, model, Location, max_speed = 30, Status_capacity = 'Empty', Destination = False):
        super().__init__(unique_id, model)

    def step(self):
        print("Hello world! I am amublance: " + str(self.unique_id) + " my node id is: " + str(self.pos))
        # grid = model.grid
        # possible_steps = grid.get_neighbors(self.pos, include_center = True)
        # print(possible_steps)

In [8]:
class Inhabitant(Agent):
    def __init__(self, unique_id, model, location, Status = '', Present_In_Ambulance = False):
        super().__init__(unique_id, model)
        self.Status = Status
        self.Present_In_Ambulance = Present_In_Ambulance

    def step(self):
        print("Hello world! I am Inhabitant: " + str(self.unique_id) + " my node id is: " + str(self.pos) + 'and I am ' + self.status)
        grid = model.grid
        possible_steps = grid.get_neighbors(self.pos, include_center = True)
        print(possible_steps)

In [16]:
# class Building():
#     def __init__(self, Location, Building_type, Building_height, Distance_from_epicenten = 1, Eearquake_intensity = 0.2):
#         self.Building_type = Building_type
#         self.Building_height = Building_height
#         self.Distance_from_epicenten = Distance_from_epicenten
#         self.Eearquake_intensity = Eearquake_intensity
#         self.Location = Location
        
#         self.VM = self.Building_type * self.Building_height * self.Distance_from_epicenten * self.Eearquake_intensity
#         self.CP = (0.7*self.VM) + 0.1
#         self.HP = (-0.15 * self.VM) +0.3
#         self.NP = (-0.55 * self.VM) +0.6
#         if self.CP+self.HP+self.NP !=1:
#             print(f'something is wrong with damage calculations, CP is {self.CP}, HP is {self.HP}, NP is {self.NP}, total is {self.CP+self.HP+self.NP}')
#         x = random.uniform(0,1)
#         statuses = [self.CP, self.HP, self.NP]
#         status_dict = {self.CP : 'The building is collapsed', self.HP : 'The building is highly damaged', self.NP: 'The building is not damaged'}
#         Sorted_statuses = sorted(statuses)
#         if Sorted_statuses[0]< x:
#             print(status_dict[Sorted_statuses[0]])
#         elif Sorted_statuses[1]< x:
#             print(status_dict[Sorted_statuses[1]])
#         elif Sorted_statuses[2]< x:
#             print(status_dict[Sorted_statuses[2]])
            
#         for i in range(80):
#             #Agents creeren
def Building_status(Building_type, Building_height, Earthquake_intensity, Epicenter, Building_node, max_distance, min_distance, distance):
        Distrance_from_epicenter = (max_distance-distance)/(max_distance-min_distance)
        VM = Building_type * Building_height * Distance_from_epicenter * Eearquake_intensity
        CP = (0.7*VM) + 0.1
        HP = (-0.15 * VM) +0.3
        NP = (-0.55 * VM) +0.6
        if CP+HP+NP !=1:
            print(f'something is wrong with damage calculations, CP is {CP}, HP is {HP}, NP is {NP}, total is {CP+HP+NP}')
        x = random.uniform(0,1)
        statuses = [CP, HP, NP]
        status_dict = {CP : 'collapsed', HP : 'highly damaged', NP: 'not damaged'}
        Sorted_statuses = sorted(statuses)
        if Sorted_statuses[0]< x:
            return(status_dict[Sorted_statuses[0]])
        elif Sorted_statuses[1]< x:
            return(status_dict[Sorted_statuses[1]])
        elif Sorted_statuses[2]< x:
            return(status_dict[Sorted_statuses[2]])

In [89]:
class Hospital():
    def __init__(self, Location, model, Hospital_id, Ambulances, Capacity, Starting_Occupancy, Working_speed = 20):
        self.Location = Location
        self.Ambulances = Ambulances
        self.Capacity = Capacity
        self.Starting_Occupancy = Starting_Occupancy
        self.Working_speed = Working_speed
        self.model = model
        self.Hospital_id = Hospital_id
        
    def printing(self):
        print('this hospital was made')
        
#         for i in range(Ambulances):
#             Ambulance_id = str(Hospital_id) + str(i)
#             a = Ambulance(Ambulance_id, self.model)
#             print(Ambulance_id)
#             #self.schedule.add(a)
#             #location = random.choice(list(self.streets))
#             #print(location)
#             #self.grid.place_agent(a, Location)

In [155]:
class MinimalModel(Model):
    def __init__(self, Hospitals = 10, Ambulances_Per_H = 10, Distance_Between_Hospitals = 50, Starting_H_Occupation = 0.2, Average_H_Capacity = 200):
        self.schedule = time.RandomActivation(self)
        self.Hospitals = Hospitals
        self.Ambulances_Per_H = Ambulances_Per_H
        self.Distance_Between_Hospitals = Distance_Between_Hospitals
        self.Starting_H_Occupation = Starting_H_Occupation
        self.Average_H_Capacity = Average_H_Capacity
        Epicenter = random.randint(1,15012)
        Earthquake_Intencity = random.choice([0.2,0.5,0.7,1.0])
        print(f'The Epicenter is located at {Epicenter}, with intencity {Earthquake_Intencity}')
        with open('street_network.data', 'rb') as file:
            self.streets = pickle.load(file)
        #self.grid = space.NetworkGrid(self.streets)
        
        #Getting the random hospital locations based on the criteria of the minimu distance
        Hospital_Location = []
        Hospital_counter = 0 #counter for the amount of hospitals
        Check_counter = 0 #counter so we don't end up in an infinte loop
        while Hospital_counter < self.Hospitals and Check_counter<1000:
            Location = random.randint(1,15012)
            if Location != Epicenter:
                if len(Hospital_Location) == 0: 
                    Hospital_Location.append(Location)
                    Hospital_counter+=1
                else:
                    Status = True # variable just created for keeping track of new hospital meets the criteria compared to the others
                    for j in Hospital_Location:
                        if nx.shortest_path_length(self.streets,Location,j,'Length') < self.Distance_Between_Hospitals:
                            Status = False
                    if Status == True:
                        Hospital_Location.append(Location)
                        Hospital_counter+=1
                    else: 
                        Check_counter+=1
        if Hospital_counter<self.Hospitals:
            print(f'The number of hospitals could not be realised under the given criteria so {i} hospitals have been created')
        
        #creating a table to keep track of the sitaution of the hospitals
        df_Hospitals = pd.DataFrame(columns=[])
        for i in Hospital_Location:
            df_Hospitals = df_Hospitals.append({'node':i, 'Total_Capacity': random.randint(round(self.Average_H_Capacity*0.8,0), round(self.Average_H_Capacity*1.2)), 'Occupancy': round(random.uniform(self.Starting_H_Occupation*0.8, self.Starting_H_Occupation*1.2),2)}, ignore_index = True)        
        df_Hospitals['Available_Capacity'] = df_Hospitals.Total_Capacity*df_Hospitals.Occupancy
        self.Hospitals = df_Hospitals
        self.Total_Hospitals = pd.DataFrame(columns=[])
        #print(sum(self.Hospitals.Total_Capacity)-sum(self.Hospitals.Available_Capacity)/sum(self.Hospitals.Total_Capacity))
            
#         Hospital_id_counter = 1
#         for i in Hospital_Locations:
#             Hospital(i, self, Hospital_id_counter, self.Ambulances_Per_H, random.randint(round(self.Average_H_Capacity*0.8,0), round(self.Average_H_Capacity*1.8)), self.Starting_H_Occupation)
#             Hospital_id_counter += 1

        #self.datacollector = DataCollector(model_reporters=model_metrics, agent_reporters=agent_metrics)

        self.running = True
        #self.datacollector.collect(self)

    def step(self):
        print("This is step: " + str(self.schedule.steps))
        self.schedule.step()
        
        #self.datacollector.collect(self)

#def count_agents(self):
    #return self.num_agents

In [ ]:
self.Total_Hospitals = Total_Hospitals.append({'Time_step': self.schedule.step, 'Average Occupancy' = ((self.Hospitals.sum('Total_Capacity')-self.Hospitals.sum('Available_Capacity'))/self.Hospitals.sum('Total_Capacity')), 'Available_Capacity'= self.Hospitals.sum(Available_Capacity), 'Used_capacity' = (self.Hospitals.sum(Total_Capacity) - self.Hospitals.sum(Available_Capacity))}, ignore_index = True)


In [153]:
R1 = MinimalModel()
R1.Hospitals

The Epicenter is located at 12273, with intencity 1.0
1931.8033074534162


,node,Total_Capacity,Occupancy,Available_Capacity
0,11541.0,211.0,0.18,37.98
1,9998.0,229.0,0.17,38.93
2,11247.0,175.0,0.18,31.50
3,10617.0,166.0,0.21,34.86
4,14411.0,195.0,0.19,37.05
5,14021.0,170.0,0.17,28.90
6,4349.0,199.0,0.17,33.83
7,8344.0,201.0,0.24,48.24
8,7615.0,196.0,0.22,43.12
9,11578.0,190.0,0.24,45.60


In [3]:
 with open('street_network.data', 'rb') as file:
            streets = pickle.load(file)

In [10]:
nx.shortest_path_length(streets,1,1475,'Length')

8611.542500000003

In [1]:
#streets.adj

In [37]:
nx.shortest_path_length(streets,50,50,'Length')

0

In [48]:
random.randint(round(200*0.8,0), round(200*1.8))

179

In [90]:
Hospital_Locations = [1,2,3,4,5]
Hospital_id_counter = 1
for i in Hospital_Locations:
    name = 'H'+str(Hospital_id_counter)
    name = Hospital(i, 'M1', Hospital_id_counter, 20, random.randint(round(200*0.8,0), round(200*1.8)), 0.2)
    Hospital_id_counter += 1

In [80]:
H1 = Hospital(i, 'M1', Hospital_id_counter, 20, random.randint(round(200*0.8,0), round(200*1.8)), 0.2)

In [92]:
print(H2.Hospital_id)

NameError: name 'H2' is not defined